# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,51.15,53,67,22.73,SH,1690425679
1,1,suva,-18.1416,178.4415,67.48,94,100,6.91,FJ,1690425730
2,2,port macquarie,-31.4333,152.9167,69.89,53,0,8.84,AU,1690425730
3,3,mainvilliers,48.4500,1.4561,63.88,87,95,7.00,FR,1690425730
4,4,ilulissat,69.2167,-51.1000,46.42,66,0,4.61,GL,1690425730


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "Humidity",
    xlabel = "Longitude",
    ylabel = "Latitude"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 95) &
                            (city_data_df["Max Temp"] > 30) &
                            (city_data_df["Wind Speed"] > 2) &
                           (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,port macquarie,-31.4333,152.9167,69.89,53,0,8.84,AU,1690425730
4,4,ilulissat,69.2167,-51.1000,46.42,66,0,4.61,GL,1690425730
40,40,forster,-32.1667,152.5167,65.37,66,0,6.40,AU,1690425734
57,57,san sperate,39.3583,9.0022,69.64,77,0,6.91,IT,1690425735
60,60,sokhumi,43.0033,41.0153,73.65,56,0,5.37,GE,1690425735
...,...,...,...,...,...,...,...,...,...,...
522,522,tamanrasset,22.7850,5.5228,84.11,10,0,12.66,DZ,1690425791
525,525,aksaray,38.3726,34.0254,68.88,24,0,3.02,TR,1690425791
526,526,goondiwindi,-28.5500,150.3167,68.29,38,0,5.70,AU,1690425791
538,538,hassi messaoud,31.6804,6.0729,91.26,38,0,4.61,DZ,1690425793


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,port macquarie,-31.4333,152.9167,69.89,53,0,8.84,AU,1690425730,
4,4,ilulissat,69.2167,-51.1000,46.42,66,0,4.61,GL,1690425730,
40,40,forster,-32.1667,152.5167,65.37,66,0,6.40,AU,1690425734,
57,57,san sperate,39.3583,9.0022,69.64,77,0,6.91,IT,1690425735,
60,60,sokhumi,43.0033,41.0153,73.65,56,0,5.37,GE,1690425735,
...,...,...,...,...,...,...,...,...,...,...,...
522,522,tamanrasset,22.7850,5.5228,84.11,10,0,12.66,DZ,1690425791,
525,525,aksaray,38.3726,34.0254,68.88,24,0,3.02,TR,1690425791,
526,526,goondiwindi,-28.5500,150.3167,68.29,38,0,5.70,AU,1690425791,
538,538,hassi messaoud,31.6804,6.0729,91.26,38,0,4.61,DZ,1690425793,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel", "apiKey" : geoapify_key, "limit":10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lon},{radius}"
    params["bias"] = f"proximity:{lat},{lon}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port macquarie - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
forster - nearest hotel: No hotel found
san sperate - nearest hotel: No hotel found
sokhumi - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
princeville - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
moose jaw - nearest hotel: No hotel found
antonio enes - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
dakhla - nearest hotel: No hotel found
pozanti - nearest hotel: No hotel found
bodo - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
oumache - nearest hotel: No hotel found
ha'il - nearest hotel: TE ORA Bag Evi Butik Hotel
rethymno - n

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,port macquarie,-31.4333,152.9167,69.89,53,0,8.84,AU,1690425730,No hotel found
4,4,ilulissat,69.2167,-51.1000,46.42,66,0,4.61,GL,1690425730,No hotel found
40,40,forster,-32.1667,152.5167,65.37,66,0,6.40,AU,1690425734,No hotel found
57,57,san sperate,39.3583,9.0022,69.64,77,0,6.91,IT,1690425735,No hotel found
60,60,sokhumi,43.0033,41.0153,73.65,56,0,5.37,GE,1690425735,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
522,522,tamanrasset,22.7850,5.5228,84.11,10,0,12.66,DZ,1690425791,No hotel found
525,525,aksaray,38.3726,34.0254,68.88,24,0,3.02,TR,1690425791,No hotel found
526,526,goondiwindi,-28.5500,150.3167,68.29,38,0,5.70,AU,1690425791,No hotel found
538,538,hassi messaoud,31.6804,6.0729,91.26,38,0,4.61,DZ,1690425793,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "Humidity",
    xlabel = "Longitude",
    ylabel = "Latitude",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)